# TUTORIAL 4 - SVM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn import datasets

from scipy import interp
from itertools import cycle

from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler, label_binarize, Normalizer
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, auc, roc_auc_score, average_precision_score

### W tym tutorialu omówiony został jeden z popularniejszych klasyfikatorów, czyli Support Vector Machine. 
#### SVM może być użyty w zadaniach regresji, jak i klasyfikacji. Głównym jego celem w klasyfikacji jest oddzielenie punktów należących do różnych klas przy użyciu N-wymiarowej hiperpłaszczyzny (N - liczba cech). Taka hiperpłaszczyzna może być wybrana na wiele różnych sposobów, natomiast SVM wybiera taką, dla której odległości między punktami nalezącymi do danej klasy są największe (tzw. maximum margin). Wyznaczona hiperpłaszczyzna jest granicą decyzyjną (decision boundary). 
#### Czym właściwie są support vectors? -> Są to punkty znajdujące się najbliżej hiperpłaszczyzny i to właśnie przy ich pomocy maksymalizujemy wspomniany wyżej margines. 
#### Warto zauważyć, że nasza hiperpłaszczyzna zależy od ilości cech, tj. dla 2 cech będzie linią, dla 3 cech będzie dwuwymiarową płaszczyzną itd. A co jeżeli klas nie da się rozdzielić hiperpłaszczyzną w N wymiarach? Wtedy do akcji wkracza tzw Kernel Trick omówiony w dalszej części

#### Słowem wstępu, warto również wyjaśnić czym właściwie jest liniowy klasyfikator. Klasyfikator liniowy charakteryzuje się tym, iż dokonuje klasyfikacji na podstawie kombinacji liniowej pewnych charakterystyk. Zatem np problem podziału danych na 2 klasy rozwiązują z wykorzystaniem linii, na 3 klasy z pomocą płaszczyzn w przypadku większej ilości klas używa wspomnianych hiperprzestrzeni.

#### W tutorialu wielokrotnie będzie obliczana wartość metryki F1, jej definicja to: F1 = (2 * precision * recall) / (precision + recall), osiąga największą wartość 1 dla idealnej precyzji oraz recall-u.
#### Inną obliczaną wartością jest ROC, przedstawia zależność True Positive Rate oraz False Positive Rate, zatem krzywa ROC dla dobrego klasyfikatora powinna mieć wypukły kształt i duże pole pod powierzchnią (area under curve - auc)
#### Zanim przejdziemy do realizacji zadań przypomnijmy, czym jest precision, a czym recall: 
#### Precision = True positive/ (True positive + False positive)
#### Recall = True positive/ (True positive + False negative)

### Zadanie 1

In [ ]:
# Importujemy zbiór iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binaryzacja wyjścia
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Dodanie losowych szumów do danych wejściowych
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# Podział zbiorów na treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=0)

# Trenowanie klasyfikatora
classifier_linear = OneVsRestClassifier(LinearSVC(random_state=random_state))
classifier_SVM = OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=random_state))

#### KERNEL TRICK - zmiana wymiarowości problemu na więcej wymiarów. Czasami dostajemy zbiory danych, których nie da się jasno sklasyfikować np. w 2 wymiarach (np. podzielić zbiór danych funkcjami liniowymi), ale w 3 wymiarach (podział płaszczyznami jest już możliwy). Wtedy rozszerza się problem na 3 wymiary lub więcej. Funkcje przekształcające tego rodzaju problemy na problemy więcej wymiarowe noszą nazwę kernel functions.

In [ ]:
def count_roc(y_score):
# Obliczenie krzywej ROC i obszaru ROC dla każdej z klas
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

# Obliczanie micro-average krzywej ROC i pola pod krzywą ROC
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Wykres krzywej ROC 
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"], color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    print("Pole pod wykresem krzywej ROC z micro-average: {0:0.2f}".format(roc_auc["micro"]))

In [ ]:
def count_precision_and_recall(y_score):
    # Dla każdej z klas liczymy precision i recall
    precision = dict()
    recall = dict()
    average_precision = dict()
    area_under_curve = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])

    # Liczymy micro-average precision i recall oraz pole pod wykresem z obliczonych wcześniej klas
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(), y_score.ravel())
    average_precision["micro"] = average_precision_score(y_test, y_score,average="micro")
    area_under_curve = auc(recall["micro"], precision["micro"])
    plt.figure()
    plt.step(recall['micro'], precision['micro'], where='post')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    print("Średnia wartość precision: {0:0.2f}".format(average_precision["micro"]))
    print("Średnia wartość pola pod wykresem precision-recall: {0:0.2f}".format(area_under_curve))

In [ ]:
classifier_kernel = OneVsRestClassifier(SVC(kernel='poly', degree=3, probability=True, random_state=random_state))
classifiers = [classifier_linear, classifier_SVM, classifier_kernel]
y_scores = []
for classifier in classifiers:
    y_scores.append(classifier.fit(X_train, y_train).decision_function(X_test))


for y_score in y_scores:
    count_roc(y_score)
    count_precision_and_recall(y_score)

### Zadanie 2

#### W zadaniu 1 wyjaśniono, czym jest kernel trick. Wiadomo, że w tym celu mogą być użyte różne funkcje, między innymi takie jak: sigmoid (często używana, jako funkcja aktywacyjna w sieciach neuronowych), radial basis function, czy też polynomial kernel. Przykładowe kernele wspierane przez SVC z biblioteki sklearn to ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’, domyślnym jest ‘rbf’.
#### Teraz dla różnych kerneli porównamy krzywe ROC i PR a także miary F-1 oraz powierzchni pod nimi.

#### Wyjaśnijmy jeszcze różnicę między ROC i PR:
##### ROC podsumowuje trade-off pomiędzy "true positive rate" oraz "false positive rate"
##### Precision-Recall podsumowuje the trade-off pomiędzy "true positive rate" oraz "positive predictive value" 
##### ROC jest istotne, gdy obserwacje pomiędzy klasami są zbalansowane, natomiast precision-recall jest odpowiednie dla niezbalansowanych zbiorów danych.

In [ ]:
k = 5
wbc = load_breast_cancer()
wbc_data = wbc.data[:200]
wbc_target = wbc.target[:200]

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
cv = StratifiedKFold(n_splits=k)

In [ ]:
def train_classfier(clf, cv, X, Y, precs, recs, fprs, tprs):
    mean_auc = 0
    mean_auc_pr = 0
    f1_mean = 0
    for i, (train, test) in enumerate(cv.split(X, Y)):
        model = clf.fit(X[train], Y[train])
        yproba = model.predict_proba(X[test])[::,1]
        pred = model.predict(X[test])
        f1 = f1_score(Y[test], pred)
        f1_mean += f1

        fpr, tpr,_ = roc_curve(Y[test],  yproba)
        fprs.append(fpr)
        tprs.append(tpr)
        curr_auc = roc_auc_score(Y[test],  yproba)
        mean_auc += curr_auc

        prec, rec, _ = precision_recall_curve(Y[test],  yproba)
        precs.append(prec)
        recs.append(rec)
        curr_auc = auc(rec,prec)
        mean_auc_pr += curr_auc
    return mean_auc_pr, mean_auc, f1_mean

In [ ]:
def plot_ROC(fprs, tprs):
    for i in range(len(fprs)):
        plt.plot(fprs[i], tprs[i])
    plt.xlabel("False Positive Rate", fontsize=15)
    plt.ylabel("True Positive Rate", fontsize=15)
    plt.show()

In [ ]:
def plot_PR(precs, recs):
    for i in range(len(precs)):
        plt.plot(precs[i], recs[i])
    plt.xlabel("Recall", fontsize=15)
    plt.ylabel("Precision", fontsize=15)
    plt.show()

In [ ]:
results_per_kernel = {}

for k_func in kernels:
    print("***** {} kernel *****\n\n".format(k_func))
    clf = SVC(kernel=k_func, probability=True)
    fprs = []
    tprs = []
    precs = []
    recs = []  
    mean_auc_pr, mean_auc, f1_mean = train_classfier(clf, cv, wbc_data, wbc_target, precs, recs, fprs, tprs)
        
    # plot ROC:
    plot_ROC(fprs, tprs)
  
    # plot Precision-Recall:
    plot_PR(precs, recs)
  
    # mean values of auc and f1
    print("Średnia wartość pola pod krzywą ROC: ", mean_auc/k)
    print("Średnia wartość pola pod krzywą PR: ", mean_auc_pr/k)
    print("Średnie F1: ", f1_mean/k)
    print("\n\n")

#### zadanie 3

#### W tym zadaniu użyjemy GridSearch-a. Przy pomocy tej klasy znajdziemy klasyfikator sparametryzowany tak aby uzyskać najlepsze dopasowanie. Następnie zwizualizujemy wyniki przy pomocy PCA

In [ ]:
def PCA_visualization(data, labels, color_map):
    df = pd.DataFrame(data) 
    normalizer = Normalizer()
    normalizer.fit(df)
    
    scaled_data = normalizer.transform(df)
    
    pca = PCA(n_components=2)
    pca.fit(scaled_data)
    
    x_pca = pca.transform(scaled_data)
    colors = np.array([color_map[x] for x in labels])
    
    plt.figure(figsize=(12, 9))
    plt.scatter(x_pca[:, 0], x_pca[:, 1], c=colors, cmap='prism')
    patches = list(map(lambda dig_col: mpatches.Patch(color=dig_col[1], label=dig_col[0]), color_map.items()))
    plt.legend(handles=patches)
    plt.show()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(wbc_data, wbc_target, train_size=.8, random_state=0)

clsf = GridSearchCV(SVC(), {'kernel': kernels}, cv=k)
clsf.fit(X_train,Y_train)
estimator = clsf.best_estimator_
print(clsf.best_params_)
pred = estimator.predict(X_test)

wbc_color_map = {
    0 : 'tab:green',
    1 : 'tab:red'
}
print(Y_test)
PCA_visualization(X_test, pred, wbc_color_map)

### ZADANIE DO WYKONANIA

#### Na zbiorze MNIST dokonaj klasyfikacji metodą SVN dla różnych kerneli ('linear', 'poly', 'rbf') oraz metodą k-NN dla k=1,3,5. Dokonaj wizualizacji otrzymanych wyników klasyfikacji przy pomocy PCA. Skomentuj otrzymane wyniki.

In [ ]:
# Ładujemy zbiór danych
mnist = fetch_openml("mnist_784", data_home="./mnist_784", cache=True)

In [ ]:
# podział zbioru na dane treningowe i testowe
n_samples = 1500
mnist_data = mnist.data[:n_samples]
mnist_target = mnist.target[:n_samples]
mnist_train_data, mnist_test_data, mnist_train_labels, mnist_test_labels = train_test_split(mnist_data, mnist_target, train_size=0.75, random_state=0)

In [ ]:
# Skorzystaj z tych kolorów w funkcji PCA_visualization()
mnist_color_map = {
    '0' : 'tab:blue',
    '1' : 'tab:orange',
    '2' : 'tab:green',
    '3' : 'tab:red',
    '4' : 'tab:purple',
    '5' : 'tab:brown',
    '6' : 'tab:pink',
    '7' : 'tab:gray',
    '8' : 'tab:olive',
    '9' : 'tab:cyan',
}

In [ ]:
def test_classifier_mnist(classifier):
    #todo
    # train classifier
    
    # print its score with test data
    
    # visualize predictions on test data using PCA_visualization()
    
    pass

In [ ]:
print('Without PCA')
print('SVM kernel: linear')
classifier = # todo
test_classifier_mnist(classifier)

# wskazówka: poeksperymentuj z parametrem gamma dla nieliniowych kerneli
print('SVM kernel: polynomial')
classifier = # todo
test_classifier_mnist(classifier)

print('SVM kernel: rbf')
classifier = # todo
test_classifier_mnist(classifier)

print('k-NN k=1')
classifier = # todo
test_classifier_mnist(classifier)

print('k-NN k=3')
classifier = # todo
test_classifier_mnist(classifier)

print('k-NN k=5')
classifier = # todo
test_classifier_mnist(classifier)

#### Wniosek: #todo

#### Dokonaj klasyfikacji ponownie, ale tym razem zredukuj liczbę wymiarów do 30 przy użyciu PCA. Skomentuj otrzymane wyniki (czy uległy poprawie i dlaczego).

In [ ]:
def test_classifier_mnist_pca(classifier):
    #todo
    # train PCA with train data
    
    # train classifier with transformed data
    
    # print its score with transformed test data
    
    # visualize predictions on test data using PCA_visualization()
    
    pass

In [ ]:
print('With PCA')
print('SVM kernel: linear')
classifier = #todo
test_classifier_mnist_pca(classifier)

print('SVM kernel: polynomial')
classifier = #todo
test_classifier_mnist_pca(classifier)

print('SVM kernel: rbf')
classifier = #todo
test_classifier_mnist_pca(classifier)

print('k-NN k=1')
classifier = #todo
test_classifier_mnist_pca(classifier)

print('k-NN k=3')
classifier = #todo
test_classifier_mnist_pca(classifier)

print('k-NN k=5')
classifier = #todo
test_classifier_mnist_pca(classifier)

#### Wniosek: #todo